In [20]:
import pandas as pd
import numpy as np
import pycountry
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2


In [21]:
data = pd.read_excel('C:/Users/admin/Desktop/Portfolio projects/Emission_project/dataset/emission_data_cleaned.xlsx')


In [22]:
data.shape

(4016, 74)

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4016 entries, 0 to 4015
Data columns (total 74 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   country                                    4016 non-null   object 
 1   year                                       4016 non-null   int64  
 2   iso_code                                   3456 non-null   object 
 3   population                                 3680 non-null   float64
 4   gdp                                        2469 non-null   float64
 5   cement_co2                                 3639 non-null   float64
 6   cement_co2_per_capita                      3607 non-null   float64
 7   co2                                        3888 non-null   float64
 8   co2_growth_abs                             3728 non-null   float64
 9   co2_growth_prct                            3728 non-null   float64
 10  co2_including_luc       

In [24]:
data_cleaned = data.dropna(subset=['country','year','co2'])

In [25]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3888 entries, 0 to 4015
Data columns (total 74 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   country                                    3888 non-null   object 
 1   year                                       3888 non-null   int64  
 2   iso_code                                   3408 non-null   object 
 3   population                                 3632 non-null   float64
 4   gdp                                        2469 non-null   float64
 5   cement_co2                                 3639 non-null   float64
 6   cement_co2_per_capita                      3607 non-null   float64
 7   co2                                        3888 non-null   float64
 8   co2_growth_abs                             3728 non-null   float64
 9   co2_growth_prct                            3728 non-null   float64
 10  co2_including_luc            

In [26]:
country_data = pd.DataFrame(data['country'].unique())

In [27]:

# Function to map country to continent
def map_country_to_continent(country_name):
    try:
        country_alpha2 = country_name_to_country_alpha2(country_name)
        continent_code = country_alpha2_to_continent_code(country_alpha2)
        continent_mapping = {
            'AF': 'Africa',
            'AS': 'Asia',
            'EU': 'Europe',
            'NA': 'North America',
            'SA': 'South America',
            'OC': 'Oceania',
            'AN': 'Antarctica'
        }
        return continent_mapping.get(continent_code, None)
    except KeyError:
        return None

# Map countries to continents
data_cleaned['continent'] = data_cleaned['country'].apply(map_country_to_continent)





C:\Users\admin\AppData\Local\Temp\ipykernel_33112\3901292087.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['continent'] = data_cleaned['country'].apply(map_country_to_continent)


In [28]:
data_cleaned[data_cleaned['continent'].isnull()].country.unique()

array(['Africa', 'Africa (GCP)', 'Asia', 'Asia (excl. China and India)',
       'Bonaire Sint Eustatius and Saba', "Cote d'Ivoire", 'Curacao',
       'Democratic Republic of Congo', 'East Timor', 'Europe',
       'Europe (GCP)', 'Europe (excl. EU-27)', 'Europe (excl. EU-28)',
       'European Union (27)', 'European Union (28)',
       'High-income countries', 'International aviation',
       'International shipping', 'Kosovo', 'Kuwaiti Oil Fires (GCP)',
       'Low-income countries', 'Lower-middle-income countries',
       'Micronesia (country)', 'Middle East (GCP)', 'Non-OECD (GCP)',
       'North America', 'North America (GCP)',
       'North America (excl. USA)', 'OECD (GCP)', 'Oceania',
       'Oceania (GCP)', 'Ryukyu Islands (GCP)', 'Saint Helena',
       'Sint Maarten (Dutch part)', 'South America',
       'South America (GCP)', 'Upper-middle-income countries', 'World'],
      dtype=object)

In [29]:
# Check for rows where continent is missing or null
missing_continent = data_cleaned[data_cleaned['continent'].isnull() | (data_cleaned['continent'] == '')]
print(missing_continent[['country', 'continent']])

     country continent
16    Africa      None
17    Africa      None
18    Africa      None
19    Africa      None
20    Africa      None
...      ...       ...
3963   World      None
3964   World      None
3965   World      None
3966   World      None
3967   World      None

[608 rows x 2 columns]


In [30]:
# Manual fixes for non-standard or grouped entities
manual_mappings = {
    'World': None,  
    'European Union': 'Europe',
    'Africa': 'Africa',
    'Africa GCP': 'Africa' ,
    'Asia (excl. China and India)': 'Asia',
    'Asia': 'Asia',
    'North America': 'North America',
    'South America': 'South America',
    'Oceania': 'Oceania',
    "Cote d'Ivoire": "Africa",
    "Curacao": "North America", 
    "Democratic Republic of Congo": "Africa",
    "Kosovo": "Europe",
    "Europe": "Europe",
    "North America (excl. USA)": "North America",
    "East Timor": "Asia",
    'Micronesia (country)': 'Oceania',
    'North America (GCP)': 'North America',
    'South America (GCP)': 'South America',
  
}

# Fill in missing continent data
data_cleaned['continent'] = data_cleaned['country'].apply(lambda x: manual_mappings.get(x, None) if x in manual_mappings else map_country_to_continent(x))



C:\Users\admin\AppData\Local\Temp\ipykernel_33112\368968565.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['continent'] = data_cleaned['country'].apply(lambda x: manual_mappings.get(x, None) if x in manual_mappings else map_country_to_continent(x))


In [31]:
data_cleaned[data_cleaned['continent'].isnull()].country.unique()

array(['Africa (GCP)', 'Bonaire Sint Eustatius and Saba', 'Europe (GCP)',
       'Europe (excl. EU-27)', 'Europe (excl. EU-28)',
       'European Union (27)', 'European Union (28)',
       'High-income countries', 'International aviation',
       'International shipping', 'Kuwaiti Oil Fires (GCP)',
       'Low-income countries', 'Lower-middle-income countries',
       'Middle East (GCP)', 'Non-OECD (GCP)', 'OECD (GCP)',
       'Oceania (GCP)', 'Ryukyu Islands (GCP)', 'Saint Helena',
       'Sint Maarten (Dutch part)', 'Upper-middle-income countries',
       'World'], dtype=object)

In [32]:
values_to_remove = ['Africa (GCP)','Europe (GCP)', 'Europe (excl. EU-27)', 'Europe (excl. EU-28)',
       'European Union (27)', 'European Union (28)',
       'High-income countries', 'International aviation',
       'International shipping','Bonaire Sint Eustatius and Saba','Kuwaiti Oil Fires (GCP)',
       'Low-income countries', 'Lower-middle-income countries',  'Middle East (GCP)', 'Non-OECD (GCP)', 'OECD (GCP)',
       'Oceania (GCP)', 'Ryukyu Islands (GCP)', 'Saint Helena',
       'Sint Maarten (Dutch part)',
       'Upper-middle-income countries','World','Asia (excl. China and India)','Europe','North America','South America','North America (GCP)',
       'North America (excl. USA)','South America (GCP)','Oceania','Africa','Asia']  

data_cleaned = data_cleaned[~data_cleaned['country'].isin(values_to_remove)]
       

In [33]:
data_cleaned.columns

Index(['country', 'year', 'iso_code', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_growth_abs', 'co2_growth_prct',
       'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'energy_per_capita', 'energy_per_gdp',
       'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_change_co2_per_capita', 'meth

In [34]:
columns_to_include = ['country', 'continent', 'year','iso_code',  'population', 'gdp','co2','co2_growth_abs', 'co2_growth_prct',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2','flaring_co2', 'gas_co2',
       'cumulative_co2',
       'cumulative_co2_including_luc'
       ]

data_cleaned = data_cleaned[columns_to_include]

In [35]:
data_cleaned.shape

(3376, 17)

In [36]:
data_cleaned.to_csv("cleaned_data.csv", index=False)